## Assignment 1 
### Anubhav a1812913

#### All Imports

In [ ]:
#import Libraries
import string
nltk.download('stopwords')  
from nltk.corpus import stopwords #Importing Stop Words
#Tokenize the words
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize #to TOkenize the words
# import lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from google.colab import drive
drive.mount('/content/drive')


### 1. Reading datasets

In [8]:
#Reading data
train_text = []
with open('drive/My Drive/NLP/Ass1/Data/reviews_train.txt', encoding='latin-1') as f:
    train_text = f.readlines()
# train_text

#reading data for testing 
test_text = []
with open('drive/My Drive/NLP/Ass1/Data/reviews_test.txt', encoding='latin-1') as f:
    test_text = f.readlines()
# test_text


#to Tokenize the words
token_words_train = [word_tokenize(text) for text in train_text]
token_words_test = [word_tokenize(text) for text in test_text]
token_words_train
token_words_test


#Seperating words into labels and training data
train_data = [i[1:-1] for i in token_words_train]
train_labels = [i[0] for i in token_words_train]
test_data = [i[1:-1] for i in token_words_test]
test_labels = [i[0] for i in token_words_test]


In [25]:
#Remove Stop words
stop_words = set(stopwords.words('english'))
rm_stp_train  = [[i for i in line if i.lower() not in stop_words]for line in train_data]
rm_stp_test = [[i for i in line if i.lower() not in stop_words]for line in test_data]

#Remove punctuation and all non-alphanumaric characters
# rm_punct_train = [[i.translate(i.maketrans('', '', string.punctuation)) for i in line ] for line in rm_stp_train]
# rm_punct_test = [[i.translate(i.maketrans('', '', string.punctuation)) for i in line] for line in rm_stp_test]
rm_punct_train = [[i for i in line if i.isalnum()] for line in rm_stp_train]
rm_punct_test = [[i for i in line if i.isalnum()] for line in rm_stp_test]

#Creating Version 1
train_version1 = rm_punct_train
test_version1 = rm_punct_test

#Performing lowercasing of words on version1 for Version 2
train_version2 = [[i.lower() for i in line]for line in train_version1]
test_version2 = [[i.lower() for i in line]for line in test_version1]

#Performing Lemmatization of words on version2 for Version 3
[lemmatizer.lemmatize(word) for word in line]

train_version3 = [[lemmatizer.lemmatize(word) for word in line]for line in train_version2]
test_version3 = [[lemmatizer.lemmatize(word) for word in line]for line in test_version2]

In [28]:
#Printing Shapes of both Versions
print(f"Shape of Version 1 Training : {len(train_version1)}\n")
print(f"Shape of Version 1 Testing : {len(test_version1)}\n")
print(f"Shape of Version 2 Training : {len(train_version2)}\n")
print(f"Shape of Version 2 Testing : {len(test_version2)}\n")
print(f"Shape of Version 3 Training : {len(train_version3)}\n")
print(f"Shape of Version 3 Testing : {len(test_version3)}\n")

Shape of Version 1 Training : 300000

Shape of Version 1 Testing : 100000

Shape of Version 2 Training : 300000

Shape of Version 2 Testing : 100000



In [ ]:
#import lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Extract lalbels
token_words_train_labels = [i[0] for i in filter_train_textUy]
# token_words_train_labels

#Remove labels form Training data
train_data = [i[1:-1] for i in token_words_train]
train_data

#Removing Stop words
stop_words = set(stopwords.words('english'))
Fiter_stop_train =[]
for line in train_data:
    fltr_line = [word for word in line if word not in stop_words]
    Fiter_stop_train.append(list(fltr_line))
# Fiter_stop_train

#Lemmatizing sentences 
lemmatized_words_train = []
for line in Fiter_stop_train:
    lemma_line = [lemmatizer.lemmatize(word) for word in line]
    lemmatized_words_train.append(list(lemma_line))
lemmatized_words_train

[['almost',
  'completely',
  'useless',
  'impossible',
  'keep',
  'ethernet',
  'cord',
  'plugged',
  'device',
  'slide',
  'use',
  'tape',
  'zip',
  'tie',
  'popsicle',
  'stick',
  'keep',
  'plugged',
  'terrible'],
 ['enough',
  'forskolin',
  'ordered',
  'reading',
  'guarantee',
  '385',
  'mg',
  'forskolin',
  'mean',
  'would',
  'take',
  'almost',
  '13',
  'pill',
  'day',
  'get',
  'scientific',
  'recommended',
  'amount',
  'need',
  'take',
  'day',
  '50',
  'mg',
  'active',
  'forskolin',
  'sorry',
  'impressed',
  'buy',
  'another',
  'product',
  'actually',
  '10',
  'mg',
  'per',
  'serving',
  'much',
  'greater'],
 ['item',
  'received',
  'item',
  'never',
  'received',
  '3',
  'email',
  'heard',
  'company',
  'refunding',
  'money',
  'sending'],
 ['great',
  'episode',
  'watch',
  'handful',
  'next',
  'gen',
  'episode',
  'best',
  'world',
  'part',
  '1',
  '2',
  'way'],
 ['time',
  'favortite',
  'book',
  'must',
  'read',
  'best',

### 2. Pre-processing

### 3. Dataset analysis

### 4. Sentiment classification using Machine Learning

### 5. Sentiment classification using VADER sentiment lexicon

### 6. (optional challenge 3 points) Combine VADER with the classifier

### 7. References